In [ ]:
## IMPORT NECESSARY LIBRARIES 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
import autograd.numpy as npg
from autograd import grad, elementwise_grad as e_grad

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, CrosshairTool, HoverTool, NumeralTickFormatter, Span
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot, layout
from bokeh.transform import cumsum
from bokeh.colors import RGB

In [ ]:
import glob
import os
import time
from datetime import datetime,date
import math
import requests
import json

In [ ]:
import yfinance as yf
import bankroll
import colorsys
# import cpi

In [ ]:
from scipy.optimize import minimize, minimize_scalar

## Defining Functions

In [ ]:
class AdamOptim():
    def __init__(self, eta=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.m_dw, self.v_dw = 0, 0
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.eta = eta
    def update(self, t, w, dw):
        self.m_dw = np.add(np.multiply(self.beta1,self.m_dw),np.multiply((1-self.beta1), dw))

        self.v_dw = np.add(np.multiply(self.beta2, self.v_dw), np.multiply((1-self.beta2),np.square(dw)))

        m_dw_corr = np.divide(self.m_dw,np.subtract((1+self.epsilon),np.power(self.beta1,t)))
        v_dw_corr = np.divide(self.v_dw,np.subtract((1+self.epsilon),np.power(self.beta2,t)))

        w = np.subtract(w, np.multiply(self.eta, np.divide(m_dw_corr, np.add(np.sqrt(v_dw_corr), self.epsilon))))
        return w

In [ ]:
def norm(x):
    x = x - np.min(x)
    x = x / np.sum(x)
    return x

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def tanh_refined(x):
    return (np.tanh(x)+1.0)/2.0

def get_weights(wts):
#     wts = sigmoid(wts)
    wts = npg.maximum(0.0, wts)
    if npg.sum(wts) != 0:
        wts = wts/npg.sum(wts)
    return wts

In [ ]:
def get_weights_v(wts):
#     wts = sigmoid(wts)
    wts = npg.maximum(0.0, wts)
    if npg.sum(wts) != 0:
        wts = wts/npg.sum(wts, axis=1, keepdims=True)
    return wts

def get_return_v(wts):
    wts = get_weights_v(wts)
    port_ret = npg.sum(log_ret_mean * wts, axis=1)
    port_ret = npg.exp(npg.multiply(port_ret, 252)) - 1
    return port_ret
    
def get_risk_v(wts):
    wts = get_weights_v(wts)
    port_sd = npg.sqrt(npg.sum(wts * npg.dot(wts, cov_mat.T), axis=1))
    return port_sd

def get_sharpe_v(wts):
    port_ret = get_return_v(wts)
    port_sd = get_risk_v(wts)
    sr = port_ret / port_sd
    return sr

def get_loss_v(risk_target=None, return_target=None, sharpe_target=None):
    weight = 1.0e3
    if risk_target is not None and return_target is not None:
        return lambda x: (weight*(npg.square(risk_target - get_risk_v(x)))+
                          (npg.square(return_target - get_return_v(x))))/(weight + 1.0)
    elif risk_target is not None and sharpe_target is not None:
        return lambda x: ((weight*npg.square(risk_target - get_risk_v(x)))+
                          (npg.square((sharpe_target - get_sharpe_v(x))/2.0)))/(weight + 1.0)
    elif risk_target is not None:
        return lambda x: ((weight*npg.square(risk_target - get_risk_v(x))) - get_return_v(x))/(weight + 1.0)
    elif return_target is not None:
        return lambda x: ((weight*npg.square(return_target - get_return_v(x))) + get_risk_v(x))/(weight + 1.0)
    return lambda x: npg.negative(get_sharpe_v(x))

In [ ]:
def filterTickers(ticks, tick_allowed):
    return [tick for tick, allowed in zip(ticks, tick_allowed) if allowed]

def get_return(wts):
    wts = np.array(wts)
#     wts = get_weights(wts)
    port_ret = npg.sum(log_ret_mean * wts)
    port_ret = np.exp(port_ret*252) - 1
    return port_ret
    
def get_risk(wts):
    wts = np.array(wts)
#     wts = get_weights(wts)
    port_sd = npg.sqrt(npg.dot(wts.T, npg.dot(cov_mat, wts)))
    return port_sd

def get_sharpe(wts):
    port_ret = get_return(wts)
    port_sd = get_risk(wts)
    sr = port_ret / port_sd
    return sr

def get_loss(risk_target=None, return_target=None, sharpe_target=None):
    weight = 1.0e3
    if risk_target is not None and return_target is not None:
        return lambda x: (weight*(npg.square(risk_target - get_risk(x)))+
                          (npg.square(return_target - get_return(x))))/(weight + 1.0)
    elif risk_target is not None and sharpe_target is not None:
        return lambda x: ((weight*npg.square(risk_target - get_risk(x)))+
                          (npg.square((sharpe_target - get_sharpe(x))/2.0)))/(weight + 1.0)
    elif risk_target is not None:
        return lambda x: ((weight*npg.square(risk_target - get_risk(x))) - get_return(x))/(weight + 1.0)
    elif return_target is not None:
        return lambda x: ((weight*npg.square(return_target - get_return(x))) + get_risk(x))/(weight + 1.0)
    return lambda x: npg.negative(get_sharpe(x))

## For adding cash

In [ ]:
def get_weights_ratio(wts_1, wts_2, ratio):
#     wts = sigmoid(wts)
    wts_1 = np.array(wts_1)
    wts_2 = np.array(wts_2)
    wts = wts_1 * ratio + wts_2 * (1.0 - ratio)
    wts = npg.maximum(0.0, wts)
    if npg.sum(wts) != 0:
        wts = wts/npg.sum(wts)
    return wts

In [ ]:
def filterTickers(ticks, tick_allowed):
    return [tick for tick, allowed in zip(ticks, tick_allowed) if allowed]

def get_return_ratio(wts_1,wts_2,ratio):
    wts = get_weights_ratio(wts_1,wts_2,ratio)
    port_ret = npg.sum(log_ret_mean * wts)
    port_ret = np.exp(port_ret*252) - 1
    return port_ret
    
def get_risk_ratio(wts_1,wts_2,ratio):
    wts = get_weights_ratio(wts_1,wts_2,ratio)
    port_sd = npg.sqrt(npg.dot(wts.T, npg.dot(cov_mat, wts)))
    return port_sd

def get_sharpe_ratio(wts_1,wts_2,ratio):
    port_ret = get_return_ratio(wts_1,wts_2,ratio)
    port_sd = get_risk_ratio(wts_1,wts_2,ratio)
    sr = port_ret / port_sd
    return sr

# def get_loss(risk_target=None, return_target=None, sharpe_target=None):
#     weight = 1.0e3
#     if risk_target is not None and return_target is not None:
#         return lambda x: (weight*(npg.square(risk_target - get_risk(x)))+
#                           (npg.square(return_target - get_return(x))))/(weight + 1.0)
#     elif risk_target is not None and sharpe_target is not None:
#         return lambda x: ((weight*npg.square(risk_target - get_risk(x)))+
#                           (npg.square((sharpe_target - get_sharpe(x))/2.0)))/(weight + 1.0)
#     elif risk_target is not None:
#         return lambda x: ((weight*npg.square(risk_target - get_risk(x))) - get_return(x))/(weight + 1.0)
#     elif return_target is not None:
#         return lambda x: ((weight*npg.square(return_target - get_return(x))) + get_risk(x))/(weight + 1.0)
#     return lambda x: npg.negative(get_sharpe(x))

In [ ]:
def get_gd_weights(weights_size, loss_fun, batch, iterations, LR, scorings, initial=None):
    # Optimize weights using gradient descent.
    if initial is not None:
        best_weights = get_weights(initial) + np.random.uniform(size=weights_size)
    else:
        best_weights = np.random.uniform(size=weights_size)
    training_gradient_fun = grad(loss_fun)
    scores = np.zeros((len(scorings), batch, iterations))
    for b in range(batch):
        if initial is not None:
            wts = get_weights(initial) + np.random.uniform(size=weights_size)
        else:
            wts = np.random.uniform(size=weights_size)
#         wts = np.random.uniform(size=weights_size)
        for i in range(iterations):
            wts = wts - training_gradient_fun(wts) * LR
            for s in range(len(scorings)):
                scores[s, b, i] = scorings[s](wts)
        if loss_fun(wts) < loss_fun(best_weights):
            best_weights = wts
    return best_weights, scores

def calculate_weights(weights_size, loss_fun_v, batch, iterations, LR, scorings, initial=None):
    adam = AdamOptim(eta=LR)
    training_gradient_fun_v = e_grad(loss_fun_v)
    scores = np.zeros((len(scorings), batch, iterations))
    wts = np.random.uniform(size=(batch, weights_size))*4
    print(wts.shape)
    for i in range(iterations):
        dw = training_gradient_fun_v(wts)
        wts = adam.update(i+1,wts, dw)
        for s in range(len(scorings)):
            scores[s, :, i] = scorings[s](wts)
    best_weights = wts[np.argmin(loss_fun_v(wts)),:]
    return best_weights, scores

In [ ]:
def plot_risk_vs_return(risk, returns, risk_title="Risk", return_title="Return"):
    fig, ax = plt.subplots(1,2)
    ax[0].plot(risk.T)
    ax[0].set_title(risk_title)
    ax[1].plot(returns.T)
    ax[1].set_title(return_title)
    
    ax[0].set_ylabel('Risk')
    ax[0].set_xlabel('Iteration')
    ax[1].set_ylabel('Return')
    ax[1].set_xlabel('Iteration')
    plt.show()

    # Portfolio composition. Min variance, max SR, max return
def plot_portfolio_composition(ticks, weights, plot_name, color_list):
#     print(ticks)
    x = dict()
    c = dict()
    for i in range(len(ticks)):
        if weights[i] >= 1.0:
            x[ticks[i]] = weights[i]
            c[ticks[i]] = color_list[i]
            x[ticks[i]+" "] = weights[i]
            c[ticks[i]+" "] = color_list[i]
        elif weights[i] > 0.001:
            x[ticks[i]] = weights[i]
            c[ticks[i]] = color_list[i]

    plot_data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'stock'})
    plot_data['angle'] = plot_data['value'] / plot_data['value'].sum() * 2 * math.pi
    
    plot_data['color'] = c.values()
    
#     print(plot_data)
#     print()
    p = figure(width=50, height=50, title=plot_name, toolbar_location=None,sizing_mode = "scale_height",
                      tools="hover", tooltips="@stock: @value{%0.1f}", x_range=(-0.5,0.5))
    p.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                   line_color="white", color='color', source=plot_data)
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    p.outline_line_color = None

    return p

## Set constants

In [ ]:
num_profiles = 5000
inflation_rate = 0.06

start_date = '2000-06-01 00:00:00'
end_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

plt.rcParams['figure.figsize'] = [15, 5]
np.random.seed(42)

In [ ]:
start_date

In [ ]:
end_date

### Define Ticks

In [ ]:
stocks = pd.read_csv("configs/stocks.csv", "\t", index_col=0, header=None).loc[:,:2]
stocks.head()

In [ ]:
stocks_limit = 100

In [ ]:
ticks = [
    "XLRE",
    "XLV",
    "FLSW",
    "ACLTX",
    "FCOM",
    "TRZBX",
    "TILWX",
    "FITLX",
    "FNIDX",
    "FNDSX",
    "SUSA",
    "IQSU", 
    "USSG",
    "SUSB",
    "SNPE",
    "SUSL",
    "EAGG",
    "WSBFX",
    "NEXTX",
    "CASH",
]

tick_allowed = [
    True,#"XLRE",
    True,#"XLV",
    True,#"FLSW",
    True,#"ACLTX",
    True,#"FCOM",
    True,#"TRZBX",
    True,#"TILWX",
    True,#"FITLX",
    True,#"FNIDX",
    True,#"FNDSX",
    True,#"SUSA",
    True,#"IQSU", 
    True,#"USSG",
    True,#"SUSB",
    True,#"SNPE",
    True,#"SUSL",
    True,#"EAGG",
    False,#"WSBFX",
    False,#"NEXTX",
    False,#"CASH",
]

tick_names = [ 
    yf.Ticker(ticker).info['longName'] for ticker in [
    "XLRE",
    "XLV",
    "FLSW",
    "ACLTX",
    "FCOM",
    "TRZBX",
    "TILWX",
    "FITLX",
    "FNIDX",
    "FNDSX",
    "SUSA",
    "IQSU", 
    "USSG",
    "SUSB",
    "SNPE",
    "SUSL",
    "EAGG",
    "WSBFX",
    "NEXTX",
]] + ["CASH"]

ticks_filtered = filterTickers(ticks, tick_allowed)
tick_name_filtered = filterTickers(tick_names, tick_allowed)
assert len(ticks) == len(tick_allowed)
assert len(ticks) == len(tick_names)

## Define Accounts

In [ ]:
path = "./configs/profiles"
FILE = open(path, 'r')
profiles = FILE.readlines()
FILE.close()
profiles = [profile.strip() for profile in profiles]

In [ ]:
profiles_names = [
    "Roth IRA",
    "Investment Account",
    "Amazon ",
    "Cash Account",
]
profiles_targets = [
    lambda x:-get_return(x),
    None,
    get_loss(),
    None
]
profiles_constraints = [
    {'type': 'eq', 'fun': lambda w: (get_risk(w) - 0.10)},
    None,
    None,
    None,
]

## Get the API Key

In [ ]:
path = "./configs/apikey"
FILE = open(path, 'r')
api_key = FILE.readline()
FILE.close()

## Get the data

In [ ]:
## MAIN BODY 

# Download historical data
start = int(time.mktime(datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timetuple()))
end = int(time.mktime(datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timetuple()))

data_dict = dict()
i = 1

for stock in ticks_filtered:
    if i % 20 == 0:
        print("Sleeping for request limit")
        time.sleep(60)
    
#     querystring = {"to": end, "symbol": stock, "from": start, "resolution": 'D'}
    querystring = f"symbol={stock}&resolution=D&from={start}&to={end}&token={api_key}"

    try:
        response = requests.request("GET", url = (f"https://finnhub.io/api/v1/stock/candle?{querystring}"))
        print(f"https://finnhub.io/api/v1/stock/candle?{querystring}")
        
        data = response.json()
        df = pd.DataFrame.from_dict(data)
        df = df.drop(columns=['s', 'h', 'l', 'o', 'v'])

        # Output time zone: Universal Time Coordinated
        df['time'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in df.t.values]

    except:
        raise ValueError("No data found for " + stock)

    i += 1
    print(i)

    # The download limit is 10 requests per minute
            
    data_dict[stock] = df




In [ ]:
# PRE PROCESS DATA TO A FRIENDLY FORMAT
data_to_concat = []
data_dict_new = {}
for key in data_dict:
    data_dict_new[key] = data_dict[key].rename(columns={"c": key})
    data_dict_new[key]['time'] = pd.to_datetime(data_dict_new[key]['time']).dt.date
    data_dict_new[key] = data_dict_new[key].set_index("time")
    data_to_concat.append(data_dict_new[key])
    
price_data = pd.concat(data_to_concat, axis=1)
price_data = price_data.loc[:,~price_data.columns.duplicated()].drop(columns='t')
price_data = price_data.sort_index()

log_ret = np.log(price_data/price_data.shift(1))

cov_mat = np.array(np.exp(log_ret.cov()*252)-1)
log_ret_mean = np.array(log_ret.mean())


In [ ]:
cov_mat.diagonal()

In [ ]:
price_data

## Load Profile Data

In [ ]:
list_of_files = glob.glob('.\profiles\*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file)
df = df[~df.Description.isna()]
df = df[['Account Number', 'Symbol', 'Current Value']]

In [ ]:
profile_makeup = pd.DataFrame(data = np.zeros((len(profiles), len(ticks))), 
                              index=profiles, 
                              columns=ticks)
profile_makeup

In [ ]:
for row_ in df.iterrows():
    if row_[1]["Symbol"] in ticks and  row_[1]["Account Number"] in profiles:
        profile_makeup.loc[row_[1]["Account Number"], row_[1]["Symbol"]] = float(row_[1]["Current Value"][1:])
    elif row_[1]["Account Number"] in profiles:
        profile_makeup.loc[row_[1]["Account Number"], "CASH"] = float(row_[1]["Current Value"][1:])
profile_makeup

## Get Inflation Data

In [ ]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0'],"startyear":"2020", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)


In [ ]:
df = pd.DataFrame.from_dict(json_data['Results']['series'][0]['data'])
df.head()
values = df["value"].to_numpy().astype(float)
inflations = (values[:-12] - values[12:])/values[12:]  
current_inflation = inflations[0]
avg_inflation = inflations.mean()

## Get Limits

### Get Max Sharpe

In [ ]:
loss = get_loss()
best_sharpe_weights = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    ).x
get_sharpe(best_sharpe_weights)

In [ ]:
# batch = 1000
# iterations = 1000
# LR=0.05

# start_time = time.time()
# best_sharpe_weights, scoring = calculate_weights(len(ticks_filtered),
#                                                         get_loss_v(), batch,
#                                                         iterations, LR,
#                                                         [get_risk_v,get_return_v])
# print("--- %s seconds ---" % (time.time() - start_time))
# plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:])
# get_sharpe(best_sharpe_weights)

### Get Risk Limits

In [ ]:
loss = get_risk
min_risk_weights = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    ).x
get_risk(min_risk_weights)

In [ ]:
# #Get Min Risk
# batch = 1000
# iterations = 1000
# LR=1.0

# start_time = time.time()
# min_risk_weights, scoring = calculate_weights(len(ticks_filtered),
#                                                 get_risk_v, batch,
#                                                 iterations, LR, 
#                                                 [get_risk_v, get_return_v])
# print("--- %s seconds ---" % (time.time() - start_time))

# # least_risk_weights, loss = get_gd_weights_adam(len(ticks_filtered), get_risk, batch, iterations, LR, [get_risk])
# # loss = loss[0,:,:]
# plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:])

# get_risk(best_sharpe_weights)

In [ ]:
loss = lambda x: -get_risk(x)
max_risk_weights = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    ).x
get_risk(max_risk_weights)

In [ ]:
# #Get Max Risk
# batch = 1000
# iterations = 1000
# LR=1.0

# loss_fun = lambda x: npg.negative(get_risk_v(x))

# start_time = time.time()
# max_risk_weights, scoring = calculate_weights(len(ticks_filtered),
#                                                 loss_fun, batch,
#                                                 iterations, LR, 
#                                                 [get_risk_v, get_return_v])
# print("--- %s seconds ---" % (time.time() - start_time))

# # least_risk_weights, loss = get_gd_weights_adam(len(ticks_filtered), get_risk, batch, iterations, LR, [get_risk])
# # loss = loss[0,:,:]
# plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:])
# get_risk(max_risk_weights)

### Get Return Range

In [ ]:
loss = lambda x: get_return(x)
min_return_weights = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    ).x
get_return(min_return_weights)

In [ ]:
# # Get Min Return
# batch = 1000
# iterations = 1000
# LR=1.0

# start_time = time.time()
# min_return_weights, scoring = calculate_weights(len(ticks_filtered),
#                                                  get_return_v, batch,
#                                                  iterations, LR,
#                                                  [get_risk_v,get_return_v])
# print("--- %s seconds ---" % (time.time() - start_time))

# # most_return_weights, loss = get_gd_weights_adam(len(ticks_filtered), 
# #                                            lambda x: -get_return(x),
# #                                            batch, iterations,
# #                                            LR, [get_risk])
# # loss = loss[0,:,:]
# plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:])
# get_risk(min_return_weights)

In [ ]:
loss = lambda x: -get_return(x)
rts = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    )
max_return_weights = rts.x
get_return(max_return_weights)
rts

In [ ]:
# # Get Max Return
# batch = 1000
# iterations = 1000
# LR=1.0
# loss_fun = lambda x: npg.negative(get_return_v(x))

# start_time = time.time()
# max_return_weights, scoring = calculate_weights(len(ticks_filtered),
#                                                  loss_fun, batch,
#                                                  iterations, LR,
#                                                  [get_risk_v,get_return_v])
# print("--- %s seconds ---" % (time.time() - start_time))

# # most_return_weights, loss = get_gd_weights_adam(len(ticks_filtered), 
# #                                            lambda x: -get_return(x),
# #                                            batch, iterations,
# #                                            LR, [get_risk])
# # loss = loss[0,:,:]
# plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:])
# get_return(max_return_weights)

## Get List of Max Returns for Spread of risks

In [ ]:
risk_min = get_risk(min_risk_weights)
risk_max = get_risk(max_return_weights)
risks_count = 50
risks_bot = np.linspace(risk_min, risk_max, risks_count, True)
risks_top = np.flip(np.linspace(risk_max, risk_min, risks_count, False))

In [ ]:
risks_top

In [ ]:
risks_bot

In [ ]:
b = 1000
i = 4000
lr = 0.1

In [ ]:
# risks[34:-13]

### Max Sharpe Spreads

In [ ]:
batch = [b]*risks_count
iterations = [i]*risks_count
LR=np.array([lr]*risks_count)
# LR[34:-13] *= 2
# LR = LR*(1.0 + 50*np.abs(0.25 - risks))
# LR[-3]*=10

best_weights_range = np.random.random(size=(risks_count, len(ticks_filtered))) 
start_time = time.time()
for r in range(risks_count):
    loss = lambda x: -get_return(x)
#     loss = get_loss(risk_target=risks[r])
#     
#     if risks[r] < get_risk(max_return_weights):
    rts = minimize(
          loss,
          np.random.random(len(ticks_filtered)),
          constraints=[
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
            {'type': 'ineq', 'fun': lambda w: -(get_risk(w) - risks_bot[r])},
#             {'type': 'ineq', 'fun': lambda w: (risks_top[r] - get_risk(w))},
          ],
          bounds=[(0., 1.) for i in range(len(ticks_filtered))]
        )
#     else:
#         rts = minimize(
#               loss,
#               np.random.random(len(ticks_filtered)),
#               constraints=[
#                 {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
#                 {'type': 'ineq', 'fun': lambda w: -(risks[r] - get_risk(w))},
#               ],
#               bounds=[(0., 1.) for i in range(len(ticks_filtered))]
#             )
        
#     print(rts.success)
    best_weights_range[r, :] = rts.x
#     print(r)
#     print(risks[r])
#     print(get_risk(best_weights_range[r, :]))
#     print(get_return(best_weights_range[r, :]))

    print("--- %s seconds ---" % (time.time() - start_time))
    
#     plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:], risk_title="Risk %f"%risks[r])

# max_return_weights = minimize(
#       loss,
#       np.random.random(len(ticks_filtered)),
#       constraints=[
#         {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
#       ],
#       bounds=[(0., 1.) for i in range(len(ticks_filtered))]
#     ).x
# get_return(max_return_weights)

In [ ]:
# return_min = get_return(min_return_weights)
# return_max = get_return(max_return_weights)
# returns_count = 50
# returns_bot = np.linspace(return_min, return_max, returns_count, False)
# returns_top = np.flip(np.linspace(return_max, return_min , returns_count, False))

# batch = [b]*risks_count
# iterations = [i]*risks_count
# LR=np.array([lr]*risks_count)
# # LR[34:-13] *= 2
# # LR = LR*(1.0 + 50*np.abs(0.25 - risks))
# # LR[-3]*=10

# returns_weights_range = np.random.random(size=(returns_count, len(ticks_filtered))) 
# start_time = time.time()
# for r in range(risks_count):
#     loss = lambda x: get_risk(x)
# #     loss = get_loss(risk_target=risks[r])
# #     
# #     if risks[r] < get_risk(max_return_weights):
#     rts = minimize(
#           loss,
#           np.random.random(len(ticks_filtered)),
#           constraints=[
#             {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
#             {'type': 'ineq', 'fun': lambda w: (get_return(w) - returns_top[r])},
#             {'type': 'ineq', 'fun': lambda w: (returns_top[r] - get_return(w))},
#           ],
#           bounds=[(0., 1.) for i in range(len(ticks_filtered))]
#         )
# #     else:
# #         rts = minimize(
# #               loss,
# #               np.random.random(len(ticks_filtered)),
# #               constraints=[
# #                 {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
# #                 {'type': 'ineq', 'fun': lambda w: -(risks[r] - get_risk(w))},
# #               ],
# #               bounds=[(0., 1.) for i in range(len(ticks_filtered))]
# #             )
        
# #     print(rts.success)
#     returns_weights_range[r, :] = rts.x
# #     print(r)
# #     print(returns_bot[r],"-", returns_top[r])
# #     print(get_risk(returns_weights_range[r, :]))
# #     print(get_return(returns_weights_range[r, :]))

#     print("--- %s seconds ---" % (time.time() - start_time))
    
# #     plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:], risk_title="Risk %f"%risks[r])

# # max_return_weights = minimize(
# #       loss,
# #       np.random.random(len(ticks_filtered)),
# #       constraints=[
# #         {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
# #       ],
# #       bounds=[(0., 1.) for i in range(len(ticks_filtered))]
# #     ).x
# # get_return(max_return_weights)

In [ ]:
# batch = [b]*risks_count
# iterations = [i]*risks_count
# LR=np.array([lr]*risks_count)
# # LR[34:-13] *= 2
# # LR = LR*(1.0 + 50*np.abs(0.25 - risks))
# # LR[-3]*=10

# best_weights_range = np.random.random(size=(risks_count, len(ticks_filtered))) 
# for r in range(risks_count):
# # for r in range(34, 50-13):
#     start_time = time.time()
#     best_weights_range[r, :], scoring = calculate_weights(len(ticks_filtered), 
#                                                                     get_loss_v(risk_target=risks[r]), 
#                                                                     batch[r],
#                                                                     iterations[r],
#                                                                     LR[r],
#                                                                     [get_risk_v,get_return_v])
#     print("--- %s seconds ---" % (time.time() - start_time))
    
#     plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:], risk_title="Risk %f"%risks[r])

### Min Sharpe Spreads

In [ ]:
# batch = [b]*risks_count
# iterations = [i]*risks_count
# LR=np.array([lr]*risks_count)
# # LR = LR*(1.0 + 50*np.abs(0.07 - risks))
# # LR[-3]*=10

# worst_weights_range = np.random.uniform(size=(risks_count, len(ticks_filtered))) 
# for r in range(risks_count):
#     start_time = time.time()
#     worst_weights_range[r, :], scoring = calculate_weights(len(ticks_filtered), 
#                                                                     get_loss_v(risk_target=risks[r], return_target=0.0), 
#                                                                     batch[r],
#                                                                     iterations[r],
#                                                                     LR[r],
#                                                                     [get_risk_v,get_return_v])
#     print("--- %s seconds ---" % (time.time() - start_time))
    
#     plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:], risk_title="Risk %f"%risks[r])

## Profile to match inflation 

In [ ]:
# batch = 1000
# iterations = 1000
# LR=0.01

# start_time = time.time()
# inflation_weights, scoring = calculate_weights(len(ticks_filtered), 
#                                            get_loss_v(return_target=avg_inflation),
#                                            batch, iterations,
#                                            LR, [get_risk_v,get_return_v])

# print("--- %s seconds ---" % (time.time() - start_time))

# plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:], return_title="Return %f"%avg_inflation)

## Build Profiles

In [ ]:
b = 1000
i = 1000
lr = 0.015
batch = [b]*len(profiles)
iterations = [i]*len(profiles)
LR=[lr]*len(profiles)

# batch[2] = 10
# iterations[2] = 200
LR[2]=0.005
target_weights = []

for i in range(len(profiles)):
    if profiles_targets[i] is not None:
        
        loss = profiles_targets[i]
        start_time = time.time()
        print(profiles_constraints[i])
        rts = minimize(
              loss,
              np.random.random(len(ticks_filtered)),
              constraints=[
                {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
                profiles_constraints[i],
              ] if profiles_constraints[i] is not None else [
                {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},],
              bounds=[(0., 1.) for i in range(len(ticks_filtered))]
            )

        print(rts.success)
        target_weights.append(rts.x)
        print(r)
        if profiles_constraints[i] is not None:
            print(0.1)
        print(get_risk(rts.x))
        print(get_return(rts.x))

        print("--- %s seconds ---" % (time.time() - start_time))
    
#         wts = profile_makeup.loc[profiles[i]].to_numpy()[:-1]
#         print(i)
#         target_weights_, scoring = calculate_weights(len(ticks_filtered),
#                                                     profiles_targets[i],
#                                                     batch[i], iterations[i],
#                                                     LR[i], [get_risk_v,get_return_v],
#                                                     initial = filterTickers(wts, tick_allowed))
#         target_weights.append(target_weights_)
# #         risk,returns = scoring[0,:,:],scoring[1,:,:]
#         plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:])
        
    else:
        
        wts = profile_makeup.loc[profiles[i]].to_numpy()
        target_weights.append(wts)


In [ ]:
profiles_targets_ratio = [
    lambda x:-get_return_ratio(filterTickers(get_weights(profile_makeup.loc[profiles[0]].to_numpy()), tick_allowed),
                               x,
                               (np.sum(filterTickers(get_weights(profile_makeup.loc[profiles[0]].to_numpy()), tick_allowed))/
                                np.sum(get_weights(profile_makeup.loc[profiles[0]].to_numpy())))),
    None,
    lambda x:-get_sharpe_ratio(filterTickers(get_weights(profile_makeup.loc[profiles[2]].to_numpy()), tick_allowed),
                               x,
                               (np.sum(filterTickers(get_weights(profile_makeup.loc[profiles[2]].to_numpy()), tick_allowed))/
                                np.sum(get_weights(profile_makeup.loc[profiles[2]].to_numpy())))),
    None
]

In [ ]:
profiles_constraints_ratio = [
    {'type': 'eq', 'fun': lambda w: (get_risk_ratio(filterTickers(get_weights(profile_makeup.loc[profiles[0]].to_numpy()), tick_allowed),
                               w,
                               (np.sum(filterTickers(get_weights(profile_makeup.loc[profiles[0]].to_numpy()), tick_allowed))/
                                np.sum(get_weights(profile_makeup.loc[profiles[0]].to_numpy())))) - 0.10)},
    None,
    None,
    None,
]

In [ ]:
(np.sum(filterTickers(get_weights(profile_makeup.loc[profiles[0]].to_numpy()), tick_allowed))/
                                np.sum(get_weights(profile_makeup.loc[profiles[0]].to_numpy())))

In [ ]:
b = 1000
i = 1000
lr = 0.015
batch = [b]*len(profiles)
iterations = [i]*len(profiles)
LR=[lr]*len(profiles)

# batch[2] = 10
# iterations[2] = 200
LR[2]=0.005
target_weights_adding = []

for i in range(len(profiles)):
    if profiles_targets_ratio[i] is not None:
        
        loss = profiles_targets_ratio[i]
        start_time = time.time()
        print(profiles_constraints[i])
        rts = minimize(
              loss,
              np.random.random(len(ticks_filtered)),
              constraints=[
                {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
                profiles_constraints_ratio[i],
              ] if profiles_constraints_ratio[i] is not None else [
                {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},],
              bounds=[(0., 1.) for i in range(len(ticks_filtered))]
            )

        print(rts.success)
        target_weights_adding.append(rts.x)
        print(r)
        if profiles_constraints_ratio[i] is not None:
            print(0.1)
        print(get_risk(rts.x))
        print(get_return(rts.x))

        print("--- %s seconds ---" % (time.time() - start_time))
    
#         wts = profile_makeup.loc[profiles[i]].to_numpy()[:-1]
#         print(i)
#         target_weights_, scoring = calculate_weights(len(ticks_filtered),
#                                                     profiles_targets_ratio[i],
#                                                     batch[i], iterations[i],
#                                                     LR[i], [get_risk_v,get_return_v],
#                                                     initial = filterTickers(wts, tick_allowed))
#         target_weights.append(target_weights_)
# #         risk,returns = scoring[0,:,:],scoring[1,:,:]
#         plot_risk_vs_return(scoring[0,:,:],scoring[1,:,:])
        
    else:
        
        wts = profile_makeup.loc[profiles[i]].to_numpy()
        target_weights_adding.append(np.zeros_like(wts))


## Display Graphs

In [ ]:
SAT = 0.5
LUM = 0.5

In [ ]:
color_list = [colorsys.hls_to_rgb(h, LUM, SAT) for h in np.linspace(0.0, 1.0, len(ticks), endpoint=False)]
color_list = [RGB(r*255,g*255,d*255) for r,g,d in color_list]
color_list_accounts = [colorsys.hls_to_rgb(h, LUM, SAT) for h in np.linspace(0.0, 1.0, len(profiles), endpoint=False)]
color_list_accounts = [RGB(r*255,g*255,d*255) for r,g,d in color_list_accounts]

In [ ]:
## PLOTS

# ===== Setup Plot ====
p = figure(
    sizing_mode = "stretch_both", 
    title="Efficient frontier. Simulations: " + str(num_profiles),
    tools='box_zoom,wheel_zoom,reset', 
    toolbar_location='above',
)
p.add_tools(CrosshairTool(line_alpha=1, line_color='lightgray', line_width=1))
p.add_tools(HoverTool(tooltips=None))

p.xaxis.axis_label = 'Volatility, or risk (standard deviation)'
p.yaxis.axis_label = 'Annual return'
p.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")

# ===== Render Funds ====
wts = np.eye(len(ticks_filtered))
risks_ = get_risk_v(wts)
returns_ = get_return_v(wts)
colors = filterTickers(color_list, tick_allowed)
renderers = []
for i in range(len(ticks_filtered)):
    c = p.circle(risks_[i],returns_[i],
             color=colors[i],
             legend_label=ticks_filtered[i], 
             name=ticks_filtered[i], 
             size=10, alpha=0.8, )
    renderers.append(c)
p.add_tools(HoverTool(renderers=renderers, tooltips=[
    ('Tick', "$name")
]))

# ===== Render Boundries ====
risk_boundry = Span(location=get_risk(min_risk_weights),
                    dimension='height', line_color='#3A5311',
                    line_width=1)
return_boundry = Span(location=get_return(max_return_weights), 
                      dimension='width', line_color='#3A5311',
                      line_width=1)
current_inf_boundry = Span(location=current_inflation, 
                      dimension='width', line_color='#03C04A',
                           line_width=1)
average_inf_boundry = Span(location=avg_inflation, 
                      dimension='width', line_color='#607D3B',
                           line_width=1)
p.renderers.extend([risk_boundry, return_boundry, current_inf_boundry, average_inf_boundry])

# ===== Render Best Sharpe Line ====
boundry =np.concatenate([
#     np.reshape(min_risk_weights,(1, len(min_risk_weights))),
    best_weights_range, 
#     np.reshape(max_risk_weights,(1, len(max_risk_weights))),
])

# c = p.circle(get_risk_v(boundry),
#          get_return_v(boundry),
#          color="#", legend_label="Max Sharpe Line", size=4)
l = p.line(
    get_risk_v(boundry),
    get_return_v(boundry), 
    color="purple",
    legend_label="Max Sharpe Line?"
    ,line_width=1)

p.add_tools(HoverTool(renderers=[l], tooltips=[
    ('Name', "Max Sharpe Line")
]))

# returns_weights_range

# # ===== Render Worst Sharpe Line ====
# boundry = np.concatenate([
#     np.reshape(min_risk_weights,(1, len(min_risk_weights))),
#     worst_weights_range, 
#     np.reshape(max_risk_weights,(1, len(max_risk_weights))),
# ])

# p.circle(get_risk_v(boundry),
#          get_return_v(boundry),
#          color="maroon", legend_label="Max Sharpe Line", size=4)
# p.line(get_risk_v(boundry),
#        get_return_v(boundry), 
#        color="maroon", line_width=2)


# ===== Render Sharpe Lines ====
p.line([0,get_return(max_return_weights)],
       [0,get_return(max_return_weights)],
       color="blue",line_width=1)
p.line([0,0.5*get_return(max_return_weights)],
       [0,get_return(max_return_weights)],
       color="blue",line_width=1)
p.line([0,(1.0/3.0)*get_return(max_return_weights)],
       [0,get_return(max_return_weights)],
       color="blue",line_width=1)

# ===== Render Existing Profile Pie Charts ====
fidelity_buy_pies = []
for i in range(len(profiles)):
    if profiles_targets_ratio[i] is not None:
        wts = get_weights(target_weights_adding[i])
        wts_point = get_weights_ratio((get_weights(profile_makeup.loc[profiles[i]].to_numpy()) 
                                 if len(target_weights_adding[i]) > len(ticks_filtered) else
                                 filterTickers(get_weights(profile_makeup.loc[profiles[i]].to_numpy()), tick_allowed)),
                                target_weights_adding[i],
                                (np.sum(filterTickers(get_weights(profile_makeup.loc[profiles[i]].to_numpy()), tick_allowed))/
                                 np.sum(get_weights(profile_makeup.loc[profiles[i]].to_numpy()))))
    else:
#         print("Elsed")
        wts = get_weights(profile_makeup.loc[profiles[i]].to_numpy())
                            
#                              get_weights(profile_makeup.loc[profiles[i]].to_numpy())
#     print("wts",wts)
    fidelity_buy_pies.append(
        plot_portfolio_composition((ticks if len(target_weights_adding[i]) > len(ticks_filtered) else ticks_filtered),
                                   wts,
                                   profiles_names[i] + " Buy",
                                   color_list))
    wts_filtered = filterTickers(wts_point, tick_allowed)
    if len(target_weights_adding[i]) == len(ticks_filtered):
        c = p.circle(get_risk(wts_point),
                     get_return(wts_point),
                     alpha=0.6,
                     color=color_list_accounts[i], 
                     name=profiles_names[i] + " Buy",
                     legend_label=profiles_names[i] + " Buy", size=15)
        renderers.append(c)
#         tooltips.append(('Profile', profiles_names[i]))

print(fidelity_buy_pies)
# p.add_tools(HoverTool(renderers=renderers, tooltips=tooltips))
# ===== Render Target Profile Pie Charts ====
fidelity_targets = []
# renderers = []
# tooltips = []
for i in range(len(target_weights)):
    fidelity_targets.append(
        plot_portfolio_composition(
            (ticks if len(target_weights[i]) > len(ticks_filtered) else ticks_filtered),
            get_weights(target_weights[i]),
            profiles_names[i] + " Target",
            color_list
        ))
    if len(target_weights[i]) == len(ticks_filtered):
        c = p.circle(get_risk(get_weights(target_weights[i])),
                     get_return(get_weights(target_weights[i])),
                     color=color_list_accounts[i],
                     alpha=0.6,
                     name=profiles_names[i] + " Target",
                     legend_label=profiles_names[i] + " Target",
                     size=15)
        renderers.append(c)
#         p.add_tools(HoverTool(renderers=[c], tooltips=[
#             ('Profile', profiles_names[i] + " Target")
#         ]))
# ===== Render Existing Profile Pie Charts ====
fidelity_pies = []
renderers = []
tooltips = []
for i in range(len(profiles)):
    wts = get_weights(profile_makeup.loc[profiles[i]].to_numpy())
#     print("wts",wts)
    fidelity_pies.append(
        plot_portfolio_composition(ticks,
                                   wts,
                                   profiles_names[i],
                                   color_list))
    wts_filtered = filterTickers(wts, tick_allowed)
    if np.sum(wts_filtered) != 0.0:
        c = p.circle(get_risk(wts_filtered), 
                     get_return(wts_filtered), 
                     color=color_list_accounts[i], 
                     name=profiles_names[i],
                     legend_label=profiles_names[i],
                     size=15)
        renderers.append(c)        

tooltips.append(('Profile', "$name"))
p.add_tools(HoverTool(renderers=renderers, tooltips=tooltips))
# ===== Adjusting Legend ====
p.legend.location = "top_left"
p.legend.visible = False


# ===== Create dashboard and open new window to show results ====
layout_ = row([ 
        column([
            p
        ], sizing_mode = "stretch_both"),
        column(fidelity_pies,
               sizing_mode = "stretch_height"), 
        column(fidelity_targets
               , sizing_mode = "stretch_height"),
        column(fidelity_buy_pies
               , sizing_mode = "stretch_height"),
    ],width = 1300, sizing_mode = "stretch_height")
    
show(layout_)